In [28]:
import numpy as np
import os, os.path
import matplotlib.image as matlib
import json

In [98]:
plane = "axial"
mouse_x = 78
mouse_y = 331
axial_slice = '81'
sagittal_slice = '322'
coronal_slice = '150'

In [99]:
npy_file = "../react-ui/src/assets/P57-16_updated/mri_rotated/indices_" + plane + "/slice_" + str(axial_slice) + ".npy"

In [100]:
npy_slice_array = np.load(npy_file)

print(np.isfortran(npy_slice_array))
print(npy_slice_array.shape)
print(npy_slice_array[300])
#print(npy_slice_array)

False
(450, 226)
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1 44 44 44 44 44 44 44 44 44 44 44
 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44
 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44
 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44
 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44 44  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0]


In [103]:
# get the block number from the numpy array
block = npy_slice_array[mouse_y][mouse_x]
print(block)

21


In [69]:
# define a vector to be used later (to get the histology coords)
# an earlier version of this notebook did +1 to the mri_slice here, need to check if this is necessary.
# need to find out the correct order of params when creating the vector (depending on the current plane)
if block == 0:
    print("block number is 0, no further calculations are necessary as there is no associated histology")

    
#if plane == "sagittal":
#  mri_vector=np.array([float(mri_slice), float(mouse_y), float(mouse_x), 1.0])
#elif plane == "coronal":
#  mri_vector=np.array([float(mri_slice), float(mouse_y), float(mouse_x), 1.0])
#elif plane == "axial":
#  mri_vector=np.array([float(mouse_x), float(mouse_y), float(mri_slice), 1.0])

mri_vector = np.array([float(sagittal_slice), float(coronal_slice), float(axial_slice), 1.0])

In [54]:
# open the matrix txt file
matrix_txt = open("../react-ui/src/assets/P57-16_updated/mri_rotated/matrices/block_" + str(block) + ".txt")
read_matrix_txt = matrix_txt.read()

In [55]:
# parse the text file and convert it to an array
matrix = read_matrix_txt.replace('\n', ' ')
matrix = matrix.split(" ")
matrix = matrix[:-1] # pops the last element in the array

In [56]:
# convert the array of strings to an array of floats
matrix_as_floats = []
for item in matrix:
    matrix_as_floats.append(float(item))

In [57]:
# divide matrix array into chuncks to form a multidimensional array
def divide_chunks(l, n):
    for i in range(0, len(l), n): 
        yield l[i:i + n]

# how many elements should have
n = 4
  
chunked_array = list(divide_chunks(matrix_as_floats, n))
print(chunked_array)

[[0.002849842303833207, -1.4686405030383833, -3.737254339395526, 1230.4328336585434], [3.9931236988629717, -0.18163689698379637, 0.07422514378423116, 123.96829658460723], [0.02615467613723934, 0.503366040657777, -0.19307029103843368, -34.53578918342316], [0.0, 0.0, 0.0, 1.0]]


In [58]:
transform = np.array(chunked_array)
print(transform)

[[ 2.84984230e-03 -1.46864050e+00 -3.73725434e+00  1.23043283e+03]
 [ 3.99312370e+00 -1.81636897e-01  7.42251438e-02  1.23968297e+02]
 [ 2.61546761e-02  5.03366041e-01 -1.93070291e-01 -3.45357892e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [59]:
blockvector = transform.dot(mri_vector)

In [60]:
print(blockvector)

[ 91.98139612 384.48804555  99.80161336   1.        ]


In [61]:
# test_transform = np.array([[0.31385434307807203, -3.664457652247729, 1.6199030926177134, 666.0080271821023],
# [-0.7744411837137424, 1.4870493065731998, 3.6535143658756977, -139.31333561581766],
# [-1.2110952755381341, -0.18593341100766062, -0.1914204859080043, 118.27995513465419],
# [0.0, 0.0, 0.0, 1.0]])

In [62]:
histology = "../react-ui/src/assets/P57-16_updated/histology"


In [63]:
folder = os.path.join(histology,"01")
slice_folder = os.path.join(folder,"slices_HE")
if os.path.exists(slice_folder):
    print(slice_folder)
else:
    print("the folder does not exist")
images = [name for name in os.listdir(slice_folder) if os.path.isfile(os.path.join(slice_folder, name))]
print(len(images))
image = None
if len(images) > 0:
    image = images[0]

../react-ui/src/assets/P57-16_updated/histology/01/slices_HE
17


In [64]:
if not image == None:
    fullPath = os.path.join(slice_folder, image)
    readImage = matlib.imread(fullPath)
    print(readImage.shape)

(851, 626, 3)
